# Comparacion de corridas (results.txt)

Lee los `results.txt` de `run_single_sim_capture` y genera tabla y graficos de cobertura, violaciones y RTB.

In [1]:
from pathlib import Path
import ast
import re
import pandas as pd
import matplotlib.pyplot as plt

def parse_final_line(line: str):
    clean = line.replace("[final]", "").strip()
    tokens = {}
    for m in re.finditer(r"(\w+)=({[^}]*}|[^\s]+)", clean):
        key, val = m.group(1), m.group(2)
        tokens[key] = val
    def get_str(key, default=""):
        return tokens.get(key, default)
    try:
        policy = get_str("policy", "unknown")
        serv_tok = get_str("serv", "0/1")
        served_str, total_str = (serv_tok.split("/", 1) + ["1"])[:2] if "/" in serv_tok else (serv_tok, "1")
        served, total = int(served_str), int(total_str)
        coverage = float(get_str("cov", "0").replace("%", "")) / 100.0
        violations = int(get_str("viol", "0"))
        rtb = int(get_str("rtb", "0"))
        serv_uav = ast.literal_eval(get_str("serv_uav", "{}")) or {}
        cov_uav = ast.literal_eval(get_str("cov_uav", "{}")) or {}
        return {
            "policy": policy,
            "served": served,
            "total": total,
            "coverage": coverage,
            "violations": violations,
            "rtb": rtb,
            "serv_uav": serv_uav,
            "cov_uav": cov_uav,
        }
    except Exception:
        return None

def load_results(paths):
    rows = []
    for p in paths:
        text = Path(p).read_text(encoding="utf-8", errors="ignore").splitlines()
        final_line = None
        for line in reversed(text):
            if "policy=" in line:
                final_line = line
                break
        if not final_line:
            print(f"[warn] sin linea final en {p}")
            continue
        parsed = parse_final_line(final_line)
        if not parsed:
            print(f"[warn] no se pudo parsear {p}: {final_line}")
            continue
        parsed["label"] = Path(p).parent.name
        parsed["path"] = str(p)
        parsed["coverage_pct"] = parsed["coverage"] * 100
        rows.append(parsed)
    return pd.DataFrame(rows)

def plot_bar(df, col, title, out):
    if df.empty:
        print(f"[warn] sin datos para {col}")
        return
    order = df.sort_values(col)
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.bar(range(len(order)), order[col], color="tab:orange")
    ax.set_xticks(range(len(order)))
    ax.set_xticklabels(order["label"], rotation=30, ha="right")
    ax.set_title(title)
    ax.set_ylabel(col)
    fig.tight_layout()
    out.parent.mkdir(parents=True, exist_ok=True)
    fig.savefig(out, dpi=150)
    plt.close(fig)
    print(f"[plot] {out}")

In [2]:
# Configura las rutas de results.txt aqui. Si se deja vacio, hace glob en results/routes/**/results.txt
FILES = []  # ejemplo: ["results/routes/poli with obstacles/results.txt", "results/routes/greedy2/results.txt"]
OUT_DIR = Path("results/routes/compare_notebook")

if FILES:
    paths = [Path(p) for p in FILES]
else:
    paths = list(Path("results/routes").rglob("results.txt"))

df = load_results(paths)
df

[warn] sin linea final en results\routes\1st run\politic with obstacles\results.txt


,policy,served,total,coverage,violations,rtb,serv_uav,cov_uav,label,path,coverage_pct
0,genetic,44,50,0.88,0,0,"{0: 12, 1: 11, 2: 11, 3: 10}","{0: 0.24, 1: 0.22, 2: 0.22, 3: 0.2}",genetica,results\routes\1st run\genetica\results.txt,88.0
1,greedy,1,50,0.02,0,4,"{0: 1, 1: 0, 2: 0, 3: 0}","{0: 0.02, 1: 0.0, 2: 0.0, 3: 0.0}",greedy,results\routes\1st run\greedy\results.txt,2.0
2,marl,18,50,0.36,0,20,"{0: 5, 1: 4, 2: 5, 3: 4}","{0: 0.1, 1: 0.08, 2: 0.1, 3: 0.08}",politic without obstacles,results\routes\1st run\politic without obstacl...,36.0
3,genetic,44,50,0.88,4,0,"{0: 12, 1: 11, 2: 11, 3: 10}","{0: 0.24, 1: 0.22, 2: 0.22, 3: 0.2}",genetica2,results\routes\genetica2\results.txt,88.0
4,greedy,1,50,0.02,0,4,"{0: 1, 1: 0, 2: 0, 3: 0}","{0: 0.02, 1: 0.0, 2: 0.0, 3: 0.0}",greedy2,results\routes\greedy2\results.txt,2.0
5,marl,15,50,0.30,4,20,"{0: 4, 1: 5, 2: 3, 3: 3}","{0: 0.08, 1: 0.1, 2: 0.06, 3: 0.06}",poli with obstacles,results\routes\poli with obstacles\results.txt,30.0
6,marl,19,50,0.38,5,20,"{0: 6, 1: 4, 2: 5, 3: 4}","{0: 0.12, 1: 0.08, 2: 0.1, 3: 0.08}",poli without obstacles,results\routes\poli without obstacles\results.txt,38.0


In [3]:
# Guardar tabla y generar plots
if not df.empty:
    OUT_DIR.mkdir(parents=True, exist_ok=True)
    table_path = OUT_DIR / "comparison.csv"
    df.to_csv(table_path, index=False)
    display(df[["label", "policy", "coverage_pct", "served", "total", "violations", "rtb"]])
    plot_bar(df, "coverage_pct", "Cobertura (%)", OUT_DIR / "coverage.png")
    plot_bar(df, "violations", "Violaciones", OUT_DIR / "violations.png")
    plot_bar(df, "rtb", "RTB events", OUT_DIR / "rtb.png")
    print(f"[tabla] {table_path}")
else:
    print("[error] no se encontraron resultados validos")

,label,policy,coverage_pct,served,total,violations,rtb
0,genetica,genetic,88.0,44,50,0,0
1,greedy,greedy,2.0,1,50,0,4
2,politic without obstacles,marl,36.0,18,50,0,20
3,genetica2,genetic,88.0,44,50,4,0
4,greedy2,greedy,2.0,1,50,0,4
5,poli with obstacles,marl,30.0,15,50,4,20
6,poli without obstacles,marl,38.0,19,50,5,20


[plot] results\routes\compare_notebook\coverage.png
[plot] results\routes\compare_notebook\violations.png
[plot] results\routes\compare_notebook\rtb.png
[tabla] results\routes\compare_notebook\comparison.csv
